# SYnDiCATE
## SAÉ 2.0 <img src="https://cdn.pixabay.com/photo/2016/11/30/09/27/hacker-1872291_960_720.jpg" width=550 align=right>

**Objectif général**  

Nous sommes une organisation secrète internationale d’activistes "white hackers" qui luttons pour la liberté.  

Nous avons écrit un texte, le MANIFESTO, qui décrit notre organisation, ses idées, ses buts, ses méthodes, ses objectifs. Jusqu’au moment où nous révélerons massivement notre existence au monde,

**le manifesto NE DOIT PAS tomber aux mains de l’ennemi.**  


Pour cette raison, le texte ne sera intégralement disponible sur aucune machine de l’organisation. A la place, il sera fragmenté en plusieurs dizaines de milliers de morceaux répartis aléatoirement sur autant de machines dans le monde.   

Pour gérer ce système nous avons besoin d'un logiciel puissant : le **SYnDICATE** ou SYstème DIstribué de ChAinage TExtuel.

Beaucoup de nos membres sont issus de vos rangs aussi nous proposons aux étudiants du BUT INFO de participer à l'élaboration du syndicate. Effectuez un bon travail et nous considérerons votre recrutement dans l'organisation.

Le but de cette SAÉ est donc d’étudier des fonctionnalités permettant de manipuler un texte réparti, sans jamais, A AUCUN MOMENT, disposer de l’intégralité du texte.  

Il en va de notre survie.

Il en va de l’avenir du monde.


## Travail et rendu

Vous travaillerez en binôme, à même ce  fichier jupyter. Vous pouvez travailler sur une installation locale, sur basthon ou bien sur google colab (méthode recommandée). Nous fournissons les codes pour charger le fichier sur google colab ou sur une installation locale.

Toutes les modalités de codage de la SAE précédente s'appliquent toujours (nommage des variables, commentaires, spécifications des fonctions). Vous êtes libres d'apporter les modifications souhaitées au code fourni, à partir du moment où **tout le code peut s'exécuter sans erreur** dans votre rendu. Vous êtes également libres d'apporter des modifications et d'ajouter vos idées.

Vous rendrez uniquement le fichier *syndicate.ipynb* en le renommant sous la forme *NOM1_NOM2.ipynb*. Les compétences évaluées sont les mêmes que pour la SAÉ1. 

La date précise de rendu vous sera communiquée via discord d'ici quelques jours.




## Fonctionnement général

Dans cette SAE, on ne manipulera pas le vrai manifesto mais un texte de substitution afin de créer le système. Ce texte a déjà été découpé en tronçons de 1 à 20 mots destinés à être répartis sur des machines. Le fichier *manifesto.txt* contient sur chaque ligne des données à stocker sur une machine. Le fichier *manifesto_short.txt* peut être utile dans la partie 2 si vous travaillez sur google colab car vous pouvez manquer de mémoire vive.

### Travail sous Basthon ou sur un serveur local 
Sous Basthon, il vous faut au préalable charger le fichier que vous souhaitez lire, en utilisant le bouton [![Image](https://i.goopics.net/rv9meb.png)](https://goopics.net/i/rv9meb)
Sélectionnez `manifesto.txt` est c'est bon: votre fichier est disponible dans basthon, comme s'il était dans votre répertoire local.

In [ ]:
#code à utiliser pour un notebook basthon ou une installation locale
def charger_manifesto(nom_fichier = "manifesto.txt"):
    """charge le fichier dont le nom est donné en paramètre
    le fichier doit se trouver dans le même répertoire
    renvoie une liste de str qui sont les lignes du fichier
    """
    f = open(nom_fichier)
    l = f.readlines()
    f.close()
    return [line.strip() for line in l]

lignes = charger_manifesto()

### Travail sous Google Colab

In [ ]:
#code à utiliser sur google colab
from google.colab import files 
import io

def charger_manifesto_colab(nom_fichier = "manifesto.txt"):
    """charge le fichier dont le nom est donné en paramètre sur google colab
    à l'exécution cliquez sur "choose file" et choisissez le bon fichier (par défaut manifesto.txt)
    renvoie une liste de str qui sont les lignes du fichier
    """

    uploaded = files.upload()
    return uploaded[nom_fichier].decode("utf-8").splitlines()


lignes = charger_manifesto_colab()

### Extraction (depuis Basthon, serveur local ou Colab)
On extrait un tableau qui contient les lignes du texte, et on affiche les 20 premières pour que vous voyiez à quoi ça ressemble.

In [ ]:
print("nombre de lignes :", len(lignes) )
print("20 premières lignes:\n")
for l in lignes[:20]:
    print(l)

Ce fichier de presque 10000 lignes (pour la version longue) contient sur sa première ligne l'adresse IP fictive d'une machine appelée FIRST. C'est l'adresse IP de la machine d'accès au réseau secret syndicate.

Ensuite, chaque ligne à partir de la troisième contient : 
- l'adresse IP d'une machine
- sa cargaison (un morceau de phrase) = l'information stockée sur cette machine
- l'adresse IP de la machine suivante

Ainsi, les machines forme une **liste chaînée** : on a une machine de départ et chaque machine connait l'adresse de la suivante. La dernière machine de la chaîne indique "LAST" pour l'adresse ip de la machine suivante (cherchez-la dans le texte !)

# Partie 0 : Fonctionnement centralisé (non sécurisé) -- algorithmes itératifs

Dans cette partie, nous utiliserons des algorithmes itératifs, exécutés depuis une machine qui centralise toutes les connaissances sur le réseau. A terme nous procéderons autrement mais nous allons écrire des fonctions qui pourront être utiles pour le test et la création du réseau.

Il va de soi que l'existence même de ce fichier *manifesto.txt* est problématique du point de vue de notre objectif et qu'il ne sert ici qu'à concevoir le système et à s'entraîner sur sa manipulation. À terme, la seule façon d'obtenir des informations sur le texte sera de procéder de façon sécurisée, de façon **chaînée**.

Néanmoins, pour se familiariser avec le fonctionnement, dans cette partie nous allons créer un annuaire des machines qui permettra d'accéder directement à toutes les machines.

Mais tout d'abord, nous créons une classe pour représenter les machines. Pour l'instant, seule la première méthode `init` est importante, c'est celle où on crée la machine et on lui attribue les trois champs.

In [ ]:
class Machine:
    """
    permet de stocker les informations d'une machine
    """
    def __init__(self, ip, cargo, next_ip):
        self.ip = ip              #str: ip de cette machine
        self.cargo = cargo        #str: informations stockées
        self.next_ip = next_ip    #str : ip de la machine suivante

    
    def executer(self, fx, arguments):
        """
        utilisé à partir de la partie 2 -- ignorer pour l'instant
        """
        return fx(self, *arguments)


Exemple : on crée un objet de la classe Machine, et on affiche ses informations ensuite. La méthode `init` est appelée indirectement, en fait on utilise le nom de la classe `Machine`pour créer un objet.

In [ ]:
m1 = Machine("103.345.232.345", "salut je suis un exemple de machine", "345.765.234.654")
#la machine m1 a été créée, elle possède trois "champs"
print("ip de la machine :",m1.ip)
print("cargaison (données stockées) :", m1.cargo)
print("ip de la machine suivante : ",m1.next_ip)

On vous donne maintenant une classe `IP_table`. On créera un unique objet de cette classe  qui simulera en quelque sorte le réseau internet pour envoyer des messages aux machines ou depuis les machines.  
Lors de la création, on donne le tableau des lignes du manifesto -- ces lignes vont être analysées (on partle de *parsing* en anglais) et les machines correspondantes créées et stockées. Vous n'avez pas forcément besoin de comprendre le détail du code ici.

In [ ]:
class IP_table:
    """permet de stocker l'ensemble des machines indexées par IP"""

    def __init__(self, tab_lignes):
        """
        crée la IP_table à partir du tableau des lignes décrivant les machines
        """
        self.table = self.charger_machines(tab_lignes)


    def charger_machines(self, tab_lignes):
        """
        crée un dictionnaire de machines à partir du tableau des lignes décrivant les machines
        """
        n = len(tab_lignes)
        first = tab_lignes[0].split('#')[1].strip() #on découpe la première ligne et on récupère l'adresse first
        table = {}

        for l in tab_lignes[2:]:
            ip, cargo, ip_next = l.split('#')[:3] 
            table[ip] = Machine(ip, cargo, ip_next)  #on découpe la ligne

        table['FIRST'] = table[first]
        del table[first]
        return table

    def get_machine(self, ip):
        """renvoie la machine qui a une ip donnée"""
        if ip in self.table:
            return self.table[ip]
 

Pour utiliser cette classe, on a juste à créer un objet de la classe `IP_table` (ci-dessous nommé `ip_table` en minuscules) et ensuite on appelle sa méthode *get_machine*.

In [ ]:
ip_table = IP_table(lignes)
f = ip_table.get_machine('FIRST')
print(f.cargo)

Il semble que la première machine ne contienne que le mot "PART". Récupérons l'adresse ip de la machine suivante et affichons, puis recommençons sur quelques machines.

In [ ]:
print(f.cargo, end = ' ')
m1 = ip_table.get_machine(f.next_ip)
print(m1.cargo, end = ' ')
m2 = ip_table.get_machine(m1.next_ip)
print(m2.cargo, end = ' ')

Pas mal, on a réussi à afficher la première phrase. Mais c'est quand même un peu laborieux. Ha mais au fait au lieu de répéter du code on peut faire une boucle non ?

**TODO** Effectuez une boucle afin d'afficher le contenu des 100 premières machines.

In [ ]:
#soluce
ip =  'FIRST'
for _ in range(100):
    m = ip_table.get_machine(ip)
    print(m.cargo, end = ' ')
    ip = m.next_ip


**TODO** Maintenant, affichez carrément le texte complet. La dernière machine de la chaîne a un champ *ip_next* qui vaut "LAST", ce qui permet de s'arrêter.

In [ ]:
#soluce
ip =  'FIRST'
while ip != 'LAST':
    m = ip_table.get_machine(ip)
    print(m.cargo, end = ' ')
    ip = m.next_ip

En fait, tout ne s'affiche pas ici car le texte est trop long.

**TODO** Utilisez le code précédent afin de compléter la fonction suivante. Il faut ajouter une espace entre les morceaux de texte pour avoir une chaîne propre.

In [ ]:
def str_texte_complet(ip_table):
    """renvoie une str qui contient le texte complet (pas de print)"""
    pass


Pour vérifier : en ajoutant un espace après chaque morceau de texte (même le dernier, pour simplifier), la longueur totale de la chaîne obtenue est de 585273.

In [ ]:
#soluce
def str_texte_complet(ip_table):
    """renvoie une str qui contient le texte complet  (pas de print)"""
    s = ""
    ip =  'FIRST'
    while ip != 'LAST':
        m = ip_table.get_machine(ip)
        s += m.cargo + " "
        ip = m.next_ip
    return s
print(len(str_texte_complet(ip_table)))

**TODO** Sur le même principe, réalisez une fonction qui renvoie le nombre de machines dont la cargaison contient un mot donné. Il est important de procéder de machine en machine, sans chercher à stocker une liste des machines dans l'ordre. Passez toujours `ip_table`en paramètre.

Nous vous laissons trouver comment utiliser une instruction python simple pour savoir si une str contient un mot.

Pour vérifier : il y a 25 machines dont la cargaison contient le mot 'cold'.

In [ ]:
#soluce
def nb_cargo_avec_mot(ip_table, mot):
    """renvoie une str qui contient le texte complet  (pas de print)"""
    nb = 0
    ip =  'FIRST'
    while ip != 'LAST':
        m = ip_table.get_machine(ip)
        if mot in m.cargo:
            nb += 1
        ip = m.next_ip
    return nb


In [ ]:
print(nb_cargo_avec_mot(ip_table, "cold"))

**Remarque** Dans le but d'étudier comment le réseau secret pourrait être réalisé, on s'interdit de réaliser une structure extérieure comme un tableau qui contiendrait la totalité des machines dans l'ordre, et on s'oblige à passer de machine en machine, toujours par ip_table. On va aussi **s'interdire de reconstituer le texte entier**.

**TODO** Ecrivez maintenant une fonction qui compte le nombre total de lettres dans le texte complet, mais sans le reconstituer. N'oubliez pas de compter un espace après chaque cargaison pour un résultat cohérent.

In [ ]:
def longueur_texte(ip_table):
    pass

**TODO** idem mais maintenant on compte les mots. Les méthodes standards de la classe **str** peuvent être utiles.

**TODO** écrivez maintenant une fonction qui renvoie le plus long mot du texte entier, toujours en procédant de machine en machine et sans reconstituer le texte entier ! En cas d'égalité de longueur, renvoyez le premier.

**TODO**  écrivez une fonction qui remplace chaque occurence d'un mot dans le texte entier, par un autre mot donné. Par exemple, remplacez toutes les occurences de "cold" par "hot", ça réchauffe.
NB : pour faire cela vous pouvez modifier directement m.cargo pour chaque machine m concernée.

**TODO**  A la fin du texte, on voudrait ajouter quelques phrases.
Ecrivez une fonction qui va ajouter une nouvelle machine, avec une IP donnée, à la fin de la liste chaînée des machines. Pour créer la machine, procédez comme expliqué au début de ce document, et mettez à jour ip_table.table qui est un dictionnaire. Pour vérifier, réutilisez les fonctions précédentes sur ce texte mis à jour.

**TODO** Ecrivez maintenant une fonction semblable, mais qui insère une nouvelle machine avec un texte donné
**juste après** la première machine qui contient un mot donné (par exemple, insérer une machine avec le texte "bla bla bla"
après la première machine dont la cargaison contient le mot "horse"). Testez. Si aucune machine ne contient ce mot, la fonction ne fait rien.

**TODO** Vous pensez à d'autres fonctions de ce type qui pourraient être utiles ? Des modifications, insertions, suppressions, etc. Codez ce que vous voulez ci-dessous.

## Partie 1 : vers un système sécurisé

Dans la partie I, le fonctionnement du système est centralisé, avec des fonctions qui peuvent obtenir directement les informations de chaque machine.
Ceci est à proscrire dans le cadre du système sécurisé que nous voulons construire. Aussi, on va s'interdire, pour une machine m, d'exécuter une fonction sur m qui va directement chercher les informations de la machine. Une machine extérieure ne doit pas pouvoir demander directement ̀̀`m.cargo` ou `m.next_ip` à une machine m.

A la place, on va demander à la machine m d'executer une fonction f sur elle-même. La différence est que la personne qui possède la machine m peut examiner le code, l'analyser et décider de l'exécuter ou non, de transmettre les informations ou non. Ce processus est plus long mais c'est un premier pas vers la sécurité.

Demander à une machine d'exécuter une fonction sur elle-même se fait se fait par l'intermédiaire de la fonction suivante.

In [ ]:
def demande_execution(ip_table, ip_machine, fx, arguments):
    """
    demande à la machine d'IP 'ip_machine' d'exécuter la fonction 'fx'
    sur la liste des arguments donnée, et de renvoyer le résultat.
    fx est une fonction de signature fx(machine, arg1, arg2, ...)
    machine represente la machine sur laquelle la fonction est appelée
    arg1, arg2... correspondent à la liste arguments
    """
    return ip_table.get_machine(ip_machine).executer(fx,arguments)

Faites attention au fait que le premier argument de la fonction sera toujours machine, qui représente la machine qui exécute la fonction (elle exécute la fonction sur elle-même et la fonction peut lui demander des informations). Pour vous aider à comprendre, voici un exemple de fonction que l'on demande à la machine FIRST d'exécuter sur elle-même.

In [ ]:
def fx_demo(machine, truc1, truc2):
    """la machine va afficher sa cargaison et les chaînes truc1 et truc2
    elle va renvoyer l'ip de la machine suivante"""
    print(machine.cargo)
    print(truc1)
    print(truc2)
    return machine.next_ip

#on demande à la machine FIRST d'exécuter la fonction sur elle-même
demande_execution(ip_table, 'FIRST', fx_demo, ["mot1", "mot2"])

Avec ce nouveau point de vue, si notre système veut demander à une machine l'ip de la machine suivante, il ne peut plus écrire directement `ipnext = m.next_ip`. Il doit envoyer à la machine une demande d'exécution d'une fonction qui renvoie l'adresse. Par exemple de la façon suivante :

In [ ]:
def get_ip_next(machine):
    """renvoie l'adresse nip de la machine suivante"""
    return machine.next_ip


In [ ]:

#on demande à la machine FIRST l'adresse de la machine suvante    
ipn = demande_execution(ip_table, 'FIRST', get_ip_next, [])
print(ipn)

**TODO** Refaites quelques fonctions de votre choix (par exemple comptage de lettres, remplacement des mots) dans ce nouveau cadre.
A chaque fois, il faut écrire la fonction principale, dont le paramètre est juste ip_table, et la fonction que l'on va demander à chaque machine d'exécuter. Le fonctionnement est semblable à la partie précédente (c'est toujours itératif et centralisé), la seule différence est dans la façon d'accéder aux données des machines.

In [ ]:

#soluce
def nb_lettres_machine(machine):
    return len(machine.cargo) + 1

def nb_lettres(ip_table):
    nb = 0
    ip =  'FIRST'
    while ip != 'LAST':

        nb += demande_execution(ip_table, ip, nb_lettres_machine,[])
        ip = demande_execution(ip_table, ip, get_ip_next, [])
    return nb
#print(nb_lettres(ip_table))


## Partie 2 : Fonctionnement sécurisé en chaîne

**Remarque** Pour cette partie, il peut être préférable d'utiliser le fichier *manifesto_short.txt* et de relever un peu les limites de la pile de récursivité de python. Si vous rencontrez des difficultés et que le système crashe, redemarrez le kernel, ne chargez pas le manifesto, gardez la mémoire pour le manifesto_short chargé ci-dessous. Il faut réexécuter les cellules comportant les définitions des classes et des fonctions utilisées.


In [ ]:
#pour éviter les erreurs sur la profondeur de récursion, on exécute ceci :
import sys
sys.setrecursionlimit(50000)

In [ ]:
#sur colab
lignes2 = charger_manifesto_colab('manifesto_short.txt')

In [ ]:
ip_table2 = IP_table(lignes2)

**Protocole SYNDICATE : système sécurisé décentralisé**  
A partir de maintenant, on va plus loin dans la sécurité. On suppose que chaque
machine ne répondra qu'aux demandes d'éxécution de la machine précédente. Pour cela, chaque machine possèdera un mot de passe permettant de demander à
la machine suivante d'exécuter une fonction, mais elle ne transmettra jamais ce mot de passe à personne.

En pratique, on ne va pas simuler ce mot de passe, mais on va écrire des codes où seulement la machine précédente a le droit de faire une demande d'exécution sur la machine suivante.

Ainsi, personne ne connaît les adresses des machines dans l'ordre. La table `ip_table` connaît les adresses de nombreuses machines (et en réalité il y aurait d'autres machines n'ayant rien à voir avec notre réseau) mais ne sait pas précisément quelles sont les machines du réseau ni dans quelle ordre elles sont chaînées, à l'exception de la machine FIRST.

**Un exemple**

On compte le nombre des lettres total du système, de façon récursive. On écrit une fonction que l'on demandera à chaque machine d'exécuter. Cette fonction va demander à la machine suivante le nombre de lettres à partir d'elle, et ajouter le nombre de lettres de la machine courante. Pour simplifier l'écriture, on passe ip_table2 en paramètre global.

In [ ]:
def total_lettres(machine) :
    """renvoie le nombre total de lettres dans toutes les cargaisons à partir de la machine donnée"""
    if machine.next_ip == 'LAST':
        lettres_suivantes = 0
    else:
        lettres_suivantes = demande_execution(ip_table2, machine.next_ip, total_lettres, [])
    return lettres_suivantes + len(machine.cargo) + 1

def nb_lettres_total_recursif(ip_table):
    """affiche le nombre total de lettres à partir de first"""
    return demande_execution(ip_table, 'FIRST', total_lettres, [])

In [ ]:
print(nb_lettres_total_recursif(ip_table2))
#verification
print(nb_lettres(ip_table2))

**TODO** En suivant le protocole SYNDICATE, réécrivez certaines des fonctions précédentes de votre choix de façon récursive et sécurisée.

A ce point, vous êtes libres d'apporter  les modifications de votre choix au code, de façon à l'améliorer, le sécuriser, avoir une simulation plus réaliste (par exemple, implémenter les mots de passe ?). Si les modifications sont trop importantes, redéfinissez les classes et fonctions utilisées dans une partie à part.